In [1]:
import json
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import classla
import json

In [95]:
import re

In [9]:
nlp = classla.Pipeline('bg', processors='tokenize,pos')

2021-09-11 22:49:23 INFO: Loading these models for language: bg (Bulgarian):
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |

2021-09-11 22:49:23 INFO: Use device: cpu
2021-09-11 22:49:23 INFO: Loading: tokenize
2021-09-11 22:49:23 INFO: Loading: pos
2021-09-11 22:49:24 INFO: Done loading processors!


In [11]:
nlp_tokenize = classla.Pipeline('bg', processors='tokenize')

2021-09-11 22:49:42 INFO: Loading these models for language: bg (Bulgarian):
| Processor | Package  |
------------------------
| tokenize  | standard |

2021-09-11 22:49:42 INFO: Use device: cpu
2021-09-11 22:49:42 INFO: Loading: tokenize
2021-09-11 22:49:42 INFO: Done loading processors!


In [4]:
def read_file_as_list_of_sentences(input_file_name):
    with open(input_file_name, "r") as input_file:
        return input_file.read().splitlines()

In [5]:
def pos_tokenize(list_of_sentences, pos_tokenize_pipeline):
    return [pos_tokenize_pipeline(sentence).to_dict()[0][0] for sentence in list_of_sentences]

In [6]:
def save_pos_tokenized_sentences_as_json(pos_tokenized_sentences, output_file_name):
    with open(output_file_name, "w") as output_file:
        json.dump(pos_tokenized_sentences, output_file) 

In [16]:
def remove_punctuation(sent):
    new_sent = ''

    for i in range(len(sent)):
        if sent[i]['text'] not in ',()"[];.?!:-':
            new_sent = new_sent + sent[i]['text'] + ' '
    
    return new_sent

In [25]:
def word2features(sent, i):
    features = {
        'word': sent[i]['text'],
        'sent_len': len(sent),
        'pos_in_sent': i,
        'upos': sent[i]['upos'],
        'xpos': sent[i]['xpos'],
        'first_word_in_sent': sent[0]['text']
    }

    if i > 0:
        features.update({
            'prev_word': sent[i-1]['text']
        })
    else:
        features.update({
            'BOS': True
        })

    if i < len(sent)-1:
        features.update({
            'next_word': sent[i+1]['text']
        })
    else:
        features.update({
            'EOS': True
        })

    return features

In [26]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [27]:
#Add support for when multiple punct signs are one after another
def word2label(sent, i):
    if i < len(sent) - 1:
        if sent[i+1]['text'] in ',()"[];.?!:-':
            return sent[i+1]['text']
        else:
            return ''
    else:
        return ''

In [28]:
def sent2labels(sent):
    return [word2label(sent, i) for i in range(len(sent))]

In [29]:
f = open('../data/Bible/processed/Bibliia_clean_dev.json', "r", encoding="utf-8")

In [30]:
f2 = open('../data/Bible/processed/Bibliia_clean_test.json', "r", encoding="utf-8")

In [31]:
data = json.load(f)

In [32]:
test_data = json.load(f2)

In [45]:
f3 = open('../data/Bible/processed/Bibliia_clean_one_sent.json', "r", encoding="utf-8")

In [46]:
f3_data = json.load(f3)

In [36]:
%%time
X_train = [sent2features(sent) for sent in data]
y_train = [sent2labels(sent) for sent in data]

X_test = [sent2features(sent) for sent in test_data]
y_test = [sent2labels(sent) for sent in test_data]

CPU times: user 264 ms, sys: 0 ns, total: 264 ms
Wall time: 265 ms


In [37]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 9.18 s, sys: 18.4 ms, total: 9.19 s
Wall time: 9.2 s


In [38]:
labels = list(crf.classes_)

In [39]:
labels

['', ',', ';', '.', '-', '!', '?', ':', '"', '(', ')']

In [40]:
labels.remove('')

In [41]:
labels

[',', ';', '.', '-', '!', '?', ':', '"', '(', ')']

In [42]:
y_pred = crf.predict(X_test)

In [44]:
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.998892383556009